   # Merge_asof
   1. 숫자나 날짜형 데이터만 가능하다.
   2. 1의 데이터가 오름차순으로 정렬이 되어있어야한다.

In [45]:
import pandas as pd
df1 = pd.DataFrame([["송중기", 92], ["권보아", 89], ["김나현", 71], ["박효신", 90]],
                    columns=["이름", "점수"])
df2 = pd.DataFrame([["A", 90], ["B", 80], ["C", 0]], columns=["학점", "점수"])
df3 = pd.DataFrame([["A", 100], ["B", 90], ["C", 80]], columns=["학점", "점수"])
df4 = pd.DataFrame([["A", 95], ["B", 85], ["C", 75]], columns=["학점", "점수"])
df5 = pd.DataFrame([[90, "lucky"], [80, "lucky"]], columns=["점수", "구분"])
t1 = ["2021-02-11", "2021-02-12", "2021-09-21", "2021-09-22", "2021-09-23"]
df6 = pd.DataFrame([t1], index=["날짜"]).T
df7 = pd.DataFrame([["2021-02-12", "설날"], ["2021-09-21", "추석"]], 
                   columns=["날짜", "내용"])
df6["날짜"] = pd.to_datetime(df6["날짜"])
df7["날짜"] = pd.to_datetime(df7["날짜"])
df8 = pd.DataFrame([["송중기", "국어", 99], ["송중기", "영어", 72], 
                    ["권보아", "국어", 92], ["권보아", "영어", 92]], 
                   columns=["이름", "과목", "점수"])
df9 = pd.DataFrame([[90, "A", "국어"], [0, "B", "국어"], [95, "A", "영어"], 
                    [0, "B", "영어"]], columns=["점수", "학점", "과목"])

# 2. 경계값의 처리

In [28]:
df1= df1.sort_values("점수")
df2= df2.sort_values("점수")
pd.merge_asof(df1, df2, allow_exact_matches = False)

# allow_exact_matches는 경계값의 처리이다. 이상과 초과..90이 B가 되었다.

,이름,점수,학점
0,김나현,71,C
1,권보아,89,B
2,박효신,90,B
3,송중기,92,A


# 3. 하한선과 상한선

In [29]:
df1= df1.sort_values("점수")
df3= df3.sort_values("점수")
pd.merge_asof(df1, df3, direction = "forward", allow_exact_matches = False)

# 100점이 A의 하한선으로 놓고 붙여준다.
# backward는 하한선
# forward는 상한선
# nearest는 중간

,이름,점수,학점
0,김나현,71,C
1,권보아,89,B
2,박효신,90,A
3,송중기,92,A


# 4. 근사값의 처리 범위

In [33]:
df1= df1.sort_values("점수")
df5= df5.sort_values("점수")
pd.merge_asof(pd.merge_asof(df1, df2), df5, tolerance = 1)

,이름,점수,학점,구분
0,김나현,71,C,NaN
1,권보아,89,B,NaN
2,박효신,90,A,lucky
3,송중기,92,A,NaN


In [39]:
df6= df6.sort_values("날짜")
df7= df7.sort_values("날짜")
pd.merge_asof(df6, df7, direction = "nearest", tolerance = pd.Timedelta('2d'))

# df6처럼 시계열이터가 쭉있는 raw데이터에서 df7처럼 마킹하고 싶은 날짜를 정하고 그 근처의 날짜만 마킹할 수 있다.

,날짜,내용
0,2021-02-11,설날
1,2021-02-12,설날
2,2021-09-21,추석
3,2021-09-22,추석
4,2021-09-23,추석


# 5. 다중요건 상황의 근사값 merge

In [41]:
df9
# 과목마다 학점이 다르다. 다중요건 

,점수,학점,과목
0,90,A,국어
1,0,B,국어
2,95,A,영어
3,0,B,영어


In [48]:
df8= df8.sort_values("점수")
df9= df9.sort_values("점수")
pd.merge_asof(df8, df9, on = "점수", by="과목")

# on은 근사시킬 열... 엄청 강력하다.

,이름,과목,점수,학점
0,송중기,영어,72,B
1,권보아,국어,92,A
2,권보아,영어,92,B
3,송중기,국어,99,A
